# AWS API HTTP Request

In [1]:
# Import Dependencies
import requests
import hashlib
import hmac
import datetime
import base64
import json

In [2]:
# Path to the image file
file_path = 'handwritten-document.jpeg'

# Read the image content
with open(file_path, 'rb') as document:
    image_bytes = document.read()

# Base64 encode the image bytes
encoded_image = base64.b64encode(image_bytes).decode('utf-8') 

In [4]:
# AWS credentials (literally never put them in clear text)
aws_access_key = 'AK...'
aws_secret_key = 'H3V...'

# AWS Service
aws_region = 'us-east-1'          # Google the list of all possible regions for your service
aws_service = 'textract'          # The service you want to use
action = 'DetectDocumentText'     # The action of the service you want to use

# Create a timestamp for the request
amz_date = datetime.datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')
datestamp = datetime.datetime.utcnow().strftime('%Y%m%d')

# Request parameters
endpoint_url = f"https://{aws_service}.{aws_region}.amazonaws.com/"
headers = {
    "Content-Type": "application/x-amz-json-1.1",
    'X-Amz-Target': f'{aws_service.capitalize()}.{action}',   # Textract.DetectDocumentText
    'Host': f'{aws_service}.{aws_region}.amazonaws.com',      # textract.us-east-1.amazonaws.com
    'X-Amz-Date': amz_date                                    # 20230904T085158Z
}

# Payload (from the Textract Documentation)
payload = {
    "Document": {
        "Bytes": encoded_image
    }
}


# Generate a canonical request
canonical_uri = '/'
canonical_querystring = ''
canonical_headers = f'content-type:{headers["Content-Type"]}\nhost:{headers["Host"]}\nx-amz-date:{amz_date}\n'
signed_headers = 'content-type;host;x-amz-date'
payload_hash = hashlib.sha256(json.dumps(payload).encode('utf-8')).hexdigest()
canonical_request = f'POST\n{canonical_uri}\n{canonical_querystring}\n{canonical_headers}\n{signed_headers}\n{payload_hash}'

# Generate a string to sign
algorithm = 'AWS4-HMAC-SHA256'
credential_scope = f'{datestamp}/{aws_region}/{aws_service}/aws4_request'
string_to_sign = f'{algorithm}\n{amz_date}\n{credential_scope}\n{hashlib.sha256(canonical_request.encode("utf-8")).hexdigest()}'

# Generate a signing key
k_secret = ('AWS4' + aws_secret_key).encode('utf-8')
k_date = hmac.new(k_secret, datestamp.encode('utf-8'), hashlib.sha256).digest()
k_region = hmac.new(k_date, aws_region.encode('utf-8'), hashlib.sha256).digest()
k_service = hmac.new(k_region, aws_service.encode('utf-8'), hashlib.sha256).digest()
signing_key = hmac.new(k_service, b'aws4_request', hashlib.sha256).digest()

# Generate the authorization header
signature = hmac.new(signing_key, string_to_sign.encode('utf-8'), hashlib.sha256).hexdigest()
authorization_header = f'{algorithm} Credential={aws_access_key}/{credential_scope}, SignedHeaders={signed_headers}, Signature={signature}'

# Add the authorization header to the request
headers['Authorization'] = authorization_header
headers['X-Amz-Date'] = amz_date

# Send the signed request
response = requests.post(endpoint_url, headers=headers, json=payload)

# check if request was successfull
if response.status_code == 200:  
    print("POST Request successful.")
else:
    print("POST Request failed.")
    print("Response:", response.text)

# create json out of response text for further use
response = response.json()

POST Request successful.


In [15]:
# Print detected text
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        print ('\033[94m' +  item["Text"] + '\033[0m')

HANDWRITING SAMPLE FORM
NAME
DATE
CITY
STATE ZIP
8-31-89
This sample of handwriting is being collected for use in testing computer recognition of hand printed numbers
Siler City, NC 27344
and letters. Please print the following characters in the boxes that appear below.
0123456789
0123456789
0123456789
0123456789
0123456789
0123456789
38
071
0755
69010
083431
38
071
0755
69010
083431
500
9534
93769
245721
64
500
9534
93769
245721
64
9494
12258
132943
82
822
9494
12258
132943
82
822
12865
167213
75
938
7570
12865
167213
75
938
7570
748850
66
376
9948
41066
748850
66
376
9948
41066
brwnjhlqysopeictkvzufdmxga
UJSKHCLMXPEOWFVNRDGYZIQBTA
JJSKHCLMXPEOWFVNRDGYZIQBTA
Please print the following text in the box below:
We, the People of the United States, in order to form a more perfect Union, establish Justice, insure domestic
Tranquility, provide for the common Defense, promote the general Welfare, and secure the Blessings of Liberty to
ourselves and our posterity, do ordain and establish this 

### JSON to Dataframe (on word level)

In [16]:
import pandas as pd

# Create lists to store extracted data
words = []
block_types = []
widths = []  # Split bounding box values into separate columns
heights = []
lefts = []
tops = []
confidences = []

# Extract words and their data from the Textract response
for block in response["Blocks"]:
    if block["BlockType"] == "WORD":
        words.append(block["Text"])
        block_types.append(block["BlockType"])
        bounding_box = block["Geometry"]["BoundingBox"]
        widths.append(bounding_box["Width"])
        heights.append(bounding_box["Height"])
        lefts.append(bounding_box["Left"])
        tops.append(bounding_box["Top"])
        confidences.append(block.get("Confidence", None))

# Create a DataFrame
data = {
    "Word": words,
    "BlockType": block_types,
    "Width": widths,
    "Height": heights,
    "Left": lefts,
    "Top": tops,
    "Confidence": confidences
}

df = pd.DataFrame(data)

# Display the DataFrame
df.head()

,Word,BlockType,Width,Height,Left,Top,Confidence
0,HANDWRITING,WORD,0.181711,0.015668,0.299809,0.090680,99.852844
1,SAMPLE,WORD,0.098359,0.015264,0.489075,0.091059,99.944000
2,FORM,WORD,0.072776,0.015002,0.594671,0.091235,99.967148
3,NAME,WORD,0.052306,0.008387,0.111683,0.139381,92.945496
4,DATE,WORD,0.047885,0.011307,0.367836,0.140057,99.914391
